# Assignment 1
> Reinforcement Learning Programming- CSCN 8020 <br>
> Name: Tai Siang Huang <br>
> ID: 9006413 

## **Problem 1 [10]**
Pick-and-Place Robot: Consider using reinforcement learning to control the motion of a robot arm in a repetitive pick-and-place task.
Design the reinforcement learning problem as an MDP, define states, actions, rewards with reasoning.

### Assuming we are controlling a 3-DOF robotic arm

### Sates
1. **Joint Angles:** `[θ₁, θ₂, θ₃]` <br>
   - Reason: Agent can learn the relationship between itself joint anglesa and space position.<br>
2. **Joint Angles' speed:** `[θ₁_v, θ₂_v, θ₃_v]`<br>
   - Reason: Agent can learn for controlling smooth and fast movement.<br>
3. **End-effector Position:** `[x, y, z]`<br>
   - Reason: Agent can determine the space's position itself.<br>
4. **Graspper State:** `[gripper_state]`<br>
   - Reason: Agent can know the gripper status.<br>
5. **Object Position:** `[x_obj, y_obj, z_obj]`<br>
   - Reason: Agent can learn where the object is and where to catch <br>
6. **Target Position:** `[x_target, y_target, z_target]`<br>
   - Reason: Agent can learn where the target is and where to place<br>
7. **Step Count:** `[step_count]`<br>
   - Reason: Agent can learn how to complete the task in the shortest time<br>

**Total Sates:** `s = [θ₁, θ₂, θ₃, θ₁_v, θ₂_v, θ₃_v, x, y, z, gripper_state, x_obj, y_obj, z_obj, x_target, y_target, z_target, step_count]`


### Actions
- `a = [Δx, Δy, Δz, g]`
- **End-effector Control:** Learning policies is more intuitive, "move towards an object" rather than "adjust a joint"
- **Gripper Action:** Gripper behavior: 1 = off, 0 = on

### Rewards
1. **Close to object:** Encourage the agent to move closer to the object.
2. **Successfully grasp the object:** A large reward will be given when the object is successfully grasped.
3. **Close to the target location:** When the object is grasped, encourage approach to the target location.
4. **Successfully drop the object in the target area:** Big rewards for completing tasks.
5. **Additional penalties:** Encourage completing tasks faster by penalizing every step, penalty for placing or dropping in the wrong place.
   
**Note:** Moving tasks use the linear positive to give reward, can control robot smooth.

In [ ]:
# ------------------------------
# Reward Function
# ------------------------------
def compute_reward(state):
    ee_pos = np.array(state["end_effector_pos"])
    obj_pos = np.array(state["object_pos"])
    tgt_pos = np.array(state["target_pos"])
    
    # Tunable parameters
    grasp_threshold = 0.05      # If EE-object distance < this, consider grasped
    place_threshold = 0.05      # If object-target distance < this, consider placed
    drop_threshold = 0.1        # If object far from both EE and target, consider dropped
    max_distance = 1.0          # For distance normalization

    # Distance calculations
    dist_to_object = np.linalg.norm(ee_pos - obj_pos)
    dist_to_target = np.linalg.norm(obj_pos - tgt_pos)
    
    # State inference
    grasped = state["gripper_state"] == 1 and dist_to_object < grasp_threshold
    placed_correctly = state["gripper_state"] == 0 and dist_to_target < place_threshold
    dropped_object_elsewhere = (
        state["gripper_state"] == 0 and
        dist_to_object > drop_threshold and
        dist_to_target > drop_threshold
    )
    
    # ------------------------------
    # Reward calculation (linear distance + task stages)
    r1 = 1 - (dist_to_object / max_distance)        # Closer to object
    r1 = max(r1, 0)
    
    r2 = 10 if grasped else 0                       # Successful grasp
    
    r3 = 0
    if grasped:
        ee_to_target = np.linalg.norm(ee_pos - tgt_pos)
        r3 = 1 - (ee_to_target / max_distance)      # While grasping, closer to target
        r3 = max(r3, 0)
    
    r4 = 20 if placed_correctly else 0              # Successful placement
    r5 = -0.1                                       # Step penalty
    r6 = -10 if dropped_object_elsewhere else 0     # Drop penalty

    total_reward = r1 + r2 + r3 + r4 + r5 + r6
    return total_reward

## Problem 2 [20]

### Iteration 1:
#### 1. Show the initial value function (V) for each state.
|   |   |
|---|---|
| 0 | 0 |
| 0 | 0 |

####  2. Perform value function updates.
Bellman Equation: $ V(s)=R(s)+γ⋅V(s')$
<br>
Due to **Initial Policy (π)** always go **up**, and default γ = 1:
- V(s1) = 5 + 0 = 5
- V(s2) = 10 + 0 = 10
- V(s3) = 1 + V(s1) = 1 + 5 = 6
- V(s4) = 2 + V(s2) = 2 + 10 = 12


####  3. Show the updated value function.
